In [1]:
import pandas as pd

In [36]:
# list of manhattan zones
manhattan = [4,24,12,13,41,45,42,43,48,50,68,79,74,75,87,88,90,125,100,103,107,113,114,116,120,127,128,151,140,137
             ,141,142,152,143,144,148,153,158,161,162,163,164,170,166,186,194,202,209,211,224,229,230,231,239,232
             ,233,234,236,237,238,243,244,246,249,261,262,263]

In [3]:
# reading taxi data
taxi = pd.read_parquet("taxi_parquet_all")

In [4]:
# first 5 rows
taxi.head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID
0,2022-12-26 16:23:47,2022-12-26 16:37:44,3.0,0.80,231,211
1,2022-12-26 16:44:40,2022-12-26 16:49:23,2.0,0.60,211,113
2,2022-12-26 16:55:48,2022-12-26 17:08:24,2.0,2.60,211,162
3,2022-12-26 16:09:49,2022-12-26 16:52:23,2.0,11.07,163,189
4,2022-12-26 16:14:49,2022-12-26 16:25:12,2.0,1.00,164,68


In [5]:
# converting drop off time to datetime
taxi["tpep_dropoff_datetime"] = pd.to_datetime(taxi["tpep_dropoff_datetime"])

In [6]:
# confirming date time conversion
taxi.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
PULocationID                      int32
DOLocationID                      int32
dtype: object

In [7]:
# dropping pickup location and time
taxi = taxi.drop(["tpep_pickup_datetime","PULocationID"],1)

/var/folders/6b/tc94k9n157g920l0_fmhzz000000gn/T/ipykernel_21388/2857857756.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  taxi = taxi.drop(["tpep_pickup_datetime","PULocationID"],1)


In [8]:
# first 5 rows
taxi.head(5)

,tpep_dropoff_datetime,passenger_count,trip_distance,DOLocationID
0,2022-12-26 16:37:44,3.0,0.80,211
1,2022-12-26 16:49:23,2.0,0.60,113
2,2022-12-26 17:08:24,2.0,2.60,162
3,2022-12-26 16:52:23,2.0,11.07,189
4,2022-12-26 16:25:12,2.0,1.00,68


In [9]:
# data frame shape
taxi.shape

(40591265, 4)

- Sources for code used to group taxi data frame
- https://sparkbyexamples.com/pandas/pandas-aggregate-functions-with-examples/?expand_article=1
- https://kanoki.org/pandas-group-dataframe-by-timeinterval
- https://sparkbyexamples.com/pandas/pandas-reset-index-examples/

In [25]:
# grouping to get the number of passengers in each location every hour
grouped_taxi = taxi.groupby(["DOLocationID", pd.Grouper(key = "tpep_dropoff_datetime", freq = "1H")]).agg({"passenger_count" 
                                                                                                           : "sum", "trip_distance" : "sum"})


In [31]:
# reseting index of grouped data frame
grouped_taxi = grouped_taxi.reset_index()

In [32]:
# shape of grouped data frame
grouped_taxi.shape

(1147880, 4)

In [33]:
# first 5 rows
grouped_taxi.head(5)

,DOLocationID,tpep_dropoff_datetime,passenger_count,trip_distance
0,1,2022-02-01 06:00:00,2.0,37.89
1,1,2022-02-01 07:00:00,5.0,44.63
2,1,2022-02-01 08:00:00,4.0,56.06
3,1,2022-02-01 09:00:00,10.0,80.56
4,1,2022-02-01 10:00:00,3.0,46.75


# Sources
- Source for how to filter data frame below
- https://datascienceparichay.com/article/filter-dataframe-on-list-of-values/
- Source on how to rename a column
- https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/

In [37]:
# filtering for rows in Manhattan
manhattan = grouped_taxi[grouped_taxi["DOLocationID"].isin(manhattan)]

In [38]:
# shape of filtered data frame
manhattan.shape

(591673, 4)

In [45]:
# first 5 rows
manhattan.head(5)

,DOLocationID,tpep_dropoff_datetime,passenger_count,trip_distance,month,dayofweek,hour
8637,4,2022-02-01 00:00:00,13.0,13.10,2,1,0
8638,4,2022-02-01 01:00:00,9.0,11.43,2,1,1
8639,4,2022-02-01 02:00:00,6.0,3.38,2,1,2
8640,4,2022-02-01 03:00:00,1.0,1.70,2,1,3
8641,4,2022-02-01 04:00:00,6.0,12.24,2,1,4


In [43]:
# creating month, day and hour columns
manhattan["month"] = manhattan["tpep_dropoff_datetime"].dt.month
manhattan["dayofweek"] = manhattan["tpep_dropoff_datetime"].dt.dayofweek
manhattan["hour"] = manhattan["tpep_dropoff_datetime"].dt.hour

/var/folders/6b/tc94k9n157g920l0_fmhzz000000gn/T/ipykernel_21388/1171177341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manhattan["month"] = manhattan["tpep_dropoff_datetime"].dt.month
/var/folders/6b/tc94k9n157g920l0_fmhzz000000gn/T/ipykernel_21388/1171177341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manhattan["dayofweek"] = manhattan["tpep_dropoff_datetime"].dt.dayofweek
/var/folders/6b/tc94k9n157g920l0_fmhzz000000gn/T/ipykernel_21388/1171177341.py:3: SettingWithCopyWarning: 
A value is 

In [44]:
# first 5 rows 
manhattan.head(5)

,DOLocationID,tpep_dropoff_datetime,passenger_count,trip_distance,month,dayofweek,hour
8637,4,2022-02-01 00:00:00,13.0,13.10,2,1,0
8638,4,2022-02-01 01:00:00,9.0,11.43,2,1,1
8639,4,2022-02-01 02:00:00,6.0,3.38,2,1,2
8640,4,2022-02-01 03:00:00,1.0,1.70,2,1,3
8641,4,2022-02-01 04:00:00,6.0,12.24,2,1,4


In [46]:
# saving filtered data frame 
manhattan.to_csv("manhattan_hourly_dropoffs.csv")

In [48]:
# renaming datetime column in taxi dataframe
manhattan.rename(columns = {"tpep_dropoff_datetime":"time"}, inplace = True)

/var/folders/6b/tc94k9n157g920l0_fmhzz000000gn/T/ipykernel_21388/1353974320.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manhattan.rename(columns = {"tpep_dropoff_datetime":"time"}, inplace = True)


In [49]:
# first 5 rows
manhattan.head(5)

,DOLocationID,time,passenger_count,trip_distance,month,dayofweek,hour
8637,4,2022-02-01 00:00:00,13.0,13.10,2,1,0
8638,4,2022-02-01 01:00:00,9.0,11.43,2,1,1
8639,4,2022-02-01 02:00:00,6.0,3.38,2,1,2
8640,4,2022-02-01 03:00:00,1.0,1.70,2,1,3
8641,4,2022-02-01 04:00:00,6.0,12.24,2,1,4


# Weather Data

Sources used to analyse weather data 
- https://www.currentresults.com/Yearly-Weather/USA/NY/New-York-City/extreme-annual-new-york-city-low-temperature.php
- https://www.currentresults.com/Yearly-Weather/USA/NY/New-York-City/extreme-annual-new-york-city-high-temperature.php#google_vignette

In [51]:
# reading weather data 
weather = pd.read_csv("weather.csv")

In [52]:
# first 5 rows
weather.head(5)

,time,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),apparent_temperature (°C),precipitation (mm),rain (mm),snowfall (cm),cloudcover (%)
0,2022-02-01T00:00,-5.8,67,-11.0,-10.6,0.0,0.0,0.0,0
1,2022-02-01T01:00,-7.1,70,-11.6,-12.0,0.0,0.0,0.0,1
2,2022-02-01T02:00,-8.6,73,-12.6,-13.7,0.0,0.0,0.0,7
3,2022-02-01T03:00,-9.8,75,-13.3,-15.0,0.0,0.0,0.0,26
4,2022-02-01T04:00,-10.8,77,-14.0,-16.0,0.0,0.0,0.0,34


In [53]:
# data frame shape
weather.shape

(8760, 9)

In [54]:
# converting time column to date time
weather["time"] = pd.to_datetime(weather["time"])

In [55]:
# creating month, day and hour columns
weather["month"] = weather["time"].dt.month
weather["dayofweek"] = weather["time"].dt.dayofweek
weather["hour"] = weather["time"].dt.hour

## Analysing Temperature

In [56]:
max_temp = weather["temperature_2m (°C)"].max()
min_temp = weather["temperature_2m (°C)"].min()
print(f"The minimum temperature is {min_temp} and the maximum temprature is {max_temp}")

The minimum temperature is -13.8 and the maximum temprature is 36.0


- According to current weather the lowest temperature in 2022 in New York City was 14 degrees celsius and the highest temprature was 36 degrees celsius, the values in the temperature column is within the permitter range

# Merging taxi and weather 


In [57]:
# merge the rows in both data frames with the same date and time
merged_df = pd.merge(weather, manhattan, on = ["time","month","dayofweek","hour"], how = "inner")

In [58]:
# shape of merged data frame
merged_df.shape

(536123, 15)

In [59]:
# first 5 rows 
merged_df.head(5)

,time,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),apparent_temperature (°C),precipitation (mm),rain (mm),snowfall (cm),cloudcover (%),month,dayofweek,hour,DOLocationID,passenger_count,trip_distance
0,2022-02-01,-5.8,67,-11.0,-10.6,0.0,0.0,0.0,0,2,1,0,4,13.0,13.10
1,2022-02-01,-5.8,67,-11.0,-10.6,0.0,0.0,0.0,0,2,1,0,13,5.0,8.91
2,2022-02-01,-5.8,67,-11.0,-10.6,0.0,0.0,0.0,0,2,1,0,24,13.0,38.40
3,2022-02-01,-5.8,67,-11.0,-10.6,0.0,0.0,0.0,0,2,1,0,41,22.0,77.32
4,2022-02-01,-5.8,67,-11.0,-10.6,0.0,0.0,0.0,0,2,1,0,42,16.0,65.17


In [60]:
# saving merged data frame as a csv file
merged_df.to_csv("taxi_weather_grouped.csv")